#Task 2: Fine Grained Image Classifications of Bird

Mounting the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


Data preparation

In [ ]:
import zipfile
zip_file = "/content/drive/My Drive/Deep Learning/task2/bird_CUB_200_2011.zip"
# extract location
extract_path = "/content/DeepLearning/task2/"
# create a ZipFile object
with zipfile.ZipFile(zip_file, 'r') as zip_obj:
  # extract all the contents of the zip file to the specified directory 
  zip_obj.extractall(extract_path)

In [ ]:
import os
import shutil
# specify the path to the directory where you want to create the folders
dir_path = "/content/DeepLearning/task2"

# specify the names of the folders you want to create
folder_name = "split_dataset"
subfolder1_name = "train"
subfolder2_name = "test"

# join the directory path and folder names
folder_path = os.path.join(dir_path, folder_name)
subfolder1_path = os.path.join(folder_path, subfolder1_name)
subfolder2_path = os.path.join(folder_path, subfolder2_name)

# check if the folders exist, and create them if they don't
if not os.path.exists(folder_path):
    os.makedirs(subfolder1_path)
    os.makedirs(subfolder2_path)
    print("Folder and subfolders created:", folder_path)
else:
    print("Folder and subfolders already exist:", folder_path)

Folder and subfolders created: /content/DeepLearning/task2/split_dataset


In [ ]:
import os
import shutil

img_dir = '/content/DeepLearning/task2/CUB_200_2011/images'
img_txt = '/content/DeepLearning/task2/CUB_200_2011/images.txt'
split_file = '/content/DeepLearning/task2/CUB_200_2011/train_test_split.txt'

test_count = 0
train_count = 0

with open(img_txt, 'r') as img_info:
  with open(split_file, 'r') as split_info:
    for line_img, line_split in zip(img_info, split_info):
      line_img = line_img.strip().split(' ')
      
      line_split = line_split.strip().split(' ')
      
      sub_dir=line_img[1].split('/')

      if line_img[0] == line_split[0]:
        if line_split[1] == '0':

          subfolder_path=os.path.join('/content/DeepLearning/task2/split_dataset/test',sub_dir[0])
          if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)
          
          path = os.path.join(img_dir, line_img[1])
          #image = tf.keras.utils.load_img(path)
          dest_path = os.path.join(subfolder_path, os.path.basename(path))
          shutil.move(path, dest_path)
          test_count += 1
        elif line_split[1] == '1':
          subfolder_path=os.path.join('/content/DeepLearning/task2/split_dataset/train',sub_dir[0])
          if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

          path = os.path.join(img_dir, line_img[1])
          #image = tf.keras.utils.load_img(path)
          dest_path = os.path.join(subfolder_path, os.path.basename(path))
          shutil.move(path, dest_path)
          train_count += 1

print(f"Moved {test_count} images to test directory")
print(f"Moved {train_count} images to train directory")

Moved 5794 images to test directory
Moved 5994 images to train directory


Loading and preprocessing the data from test and train directories. Image datasets are generated

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#normalize to scale between 0 and 1
train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.10)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_generator =  test_datagen.flow_from_directory('/content/DeepLearning/task2/split_dataset/test',
                                             target_size= (224, 224),
                                             class_mode='categorical',
                                             color_mode='rgb', batch_size=32, 
                                             shuffle=False)


train_generator =  train_datagen.flow_from_directory('/content/DeepLearning/task2/split_dataset/train',
                                             target_size= (224, 224),
                                             class_mode='categorical',
                                             color_mode='rgb', batch_size=32,shuffle=True)

valid_generator = train_datagen.flow_from_directory('/content/DeepLearning/task2/split_dataset/train',
                                             target_size= (224, 224), 
                                             class_mode='categorical',
                                             color_mode='rgb', batch_size=32, subset='validation',shuffle=True)





Found 5794 images belonging to 200 classes.
Found 5994 images belonging to 200 classes.
Found 594 images belonging to 200 classes.


Model Building

In [23]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping

IMAGE_SIZE = [224, 224]
base_model = InceptionV3( input_shape=IMAGE_SIZE + [3],  weights='imagenet',include_top=False,)

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(200, activation='softmax')(x)

In [28]:
#early stopping with validation loss to onitor
es = EarlyStopping(monitor="val_loss",patience=10,verbose=0,mode="auto")
checkpoint_filepath = "/content/drive/MyDrive/Deep Learning/task2/task2_checkpoints/Inception_net/adam/inception_net_checkpoints.{epoch:03d}"

# Create a ModelCheckpoint callback to save the model at each checkpoint
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=False)

In [29]:

# Define the function to adjust the learning rate
# define the learning rate schedule function based on accuracy
def lr_schedule(epoch, accuracy):
    if accuracy < 0.7:
        return 0.0001
    
    else:
        return 0.00001

# Create a LearningRateScheduler callback to adjust the learning rate at each epoch
lr_scheduler_callback = LearningRateScheduler(lr_schedule)

Model Compiling and training

In [17]:
#compile and fit the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, callbacks = [es, model_checkpoint_callback, lr_scheduler_callback], validation_data=valid_generator, batch_size = 32, epochs = 30)
#model saved in the path given
model.save('/content/drive/MyDrive/Deep Learning/task2/task2_inception_net_adam_model_final1.h5')

Epoch 1/30
188/188 [==============================] - ETA: 0s - loss: 4.2878 - accuracy: 0.1505

188/188 [==============================] - 175s 751ms/step - loss: 4.2878 - accuracy: 0.1505 - val_loss: 2.4468 - val_accuracy: 0.3822 - lr: 1.0000e-04
Epoch 2/30
188/188 [==============================] - ETA: 0s - loss: 1.9534 - accuracy: 0.5043

188/188 [==============================] - 137s 730ms/step - loss: 1.9534 - accuracy: 0.5043 - val_loss: 1.1490 - val_accuracy: 0.6886 - lr: 1.0000e-04
Epoch 3/30
188/188 [==============================] - ETA: 0s - loss: 1.0842 - accuracy: 0.7030

188/188 [==============================] - 137s 727ms/step - loss: 1.0842 - accuracy: 0.7030 - val_loss: 0.6393 - val_accuracy: 0.8266 - lr: 1.0000e-04
Epoch 4/30
188/188 [==============================] - ETA: 0s - loss: 0.6956 - accuracy: 0.8110

188/188 [==============================] - 185s 987ms/step - loss: 0.6956 - accuracy: 0.8110 - val_loss: 0.4007 - val_accuracy: 0.8906 - lr: 1.0000e-04
Epoch 5/30
188/188 [==============================] - ETA: 0s - loss: 0.4556 - accuracy: 0.8745

...

188/188 [==============================] - 137s 732ms/step - loss: 0.0596 - accuracy: 0.9823 - val_loss: 0.0323 - val_accuracy: 0.9949 - lr: 1.0000e-04
Epoch 27/30
188/188 [==============================] - ETA: 0s - loss: 0.0661 - accuracy: 0.9820

188/188 [==============================] - 137s 732ms/step - loss: 0.0661 - accuracy: 0.9820 - val_loss: 0.0767 - val_accuracy: 0.9798 - lr: 1.0000e-04
Epoch 28/30
188/188 [==============================] - ETA: 0s - loss: 0.0754 - accuracy: 0.9781

188/188 [==============================] - 139s 741ms/step - loss: 0.0754 - accuracy: 0.9781 - val_loss: 0.0461 - val_accuracy: 0.9815 - lr: 1.0000e-04
Epoch 29/30
188/188 [==============================] - ETA: 0s - loss: 0.0712 - accuracy: 0.9790

188/188 [==============================] - 140s 745ms/step - loss: 0.0712 - accuracy: 0.9790 - val_loss: 0.0341 - val_accuracy: 0.9865 - lr: 1.0000e-04
Epoch 30/30
188/188 [==============================] - ETA: 0s - loss: 0.0564 - accuracy: 0.9847

188/188 [==============================] - 139s 738ms/step - loss: 0.0564 - accuracy: 0.9847 - val_loss: 0.0145 - val_accuracy: 0.9966 - lr: 1.0000e-04


In [18]:
results = model.evaluate(test_generator, verbose=0)
print(results)
print(f"Test Accuracy: {np.round(results[1] * 100,2)}%")

[1.6889314651489258, 0.6693130731582642]
Test Accuracy: 66.93%


In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score
batch_size = 8
# generate the predicted classes for the test data using predict_generator
y_pred = model.predict(test_generator, steps=test_generator.samples//batch_size)

# get the true classes for the test data
y_true = test_generator.classes

# convert the predicted classes from one-hot encoding to class indices
y_pred_classes = np.argmax(y_pred, axis=1)

#compute accuracy score
acc = accuracy_score(y_true, y_pred_classes)
print("accuracy score:", acc)
print("F1 score: ", f1_score(y_true, y_pred_classes, average='weighted') )
print("precision: ",precision_score(y_true, y_pred_classes, average = 'weighted'))

# compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# compute the classification report
class_report = classification_report(y_true, y_pred_classes)
print("Classification Report:\n", class_report)


181/724 [======>.......................] - ETA: 1:33

724/724 [==============================] - 31s 43ms/step
accuracy score: 0.669313082499137
F1 score:  0.6657070601248848
precision:  0.7003359267563795
Confusion Matrix:
 [[22  1  0 ...  0  0  0]
 [ 1 18  0 ...  0  0  0]
 [ 5  4 10 ...  0  0  0]
 ...
 [ 0  0  0 ... 20  0  0]
 [ 0  0  0 ...  1 14  0]
 [ 0  0  0 ...  0  0 24]]
Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.73      0.62        30
           1       0.78      0.60      0.68        30
           2       1.00      0.36      0.53        28
           3       0.62      0.60      0.61        30
           4       0.80      0.86      0.83        14
           5       1.00      0.82      0.90        11
           6       0.95      0.91      0.93        23
           7       0.80      0.67      0.73        18
           8       0.61      0.38      0.47        29
           9       0.83      0.83      0.83        30
          10       0.64      0.30      0.41        30
